In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Dataset
import os
import pickle
import numpy as np

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=False):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

       
        self.downsample = downsample
        if downsample:
            self.skip_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            self.skip_bn = nn.BatchNorm2d(out_channels)
        else:
            self.skip_conv = None  # Ensure it's defined

    def forward(self, x):
        identity = x

        
        if self.downsample:
            identity = self.skip_bn(self.skip_conv(x))

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity  
        return F.relu(out)

In [3]:
class CustomResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomResNet, self).__init__()
        
        # Initial Convolution
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # Residual layers with variable channels
        self.layer1 = self._make_layer(64, 64, num_blocks=2, stride=1)  # No downsampling
        self.layer2 = self._make_layer(64, 128, num_blocks=2, stride=2)  # Downsampling
        self.layer3 = self._make_layer(128, 256, num_blocks=2, stride=2)  # Downsampling
        self.layer4 = self._make_layer(256, 512, num_blocks=2, stride=2)  # Downsampling

        # Average Pooling (kernel size = 2, stride = 2)
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer
        self.fc = nn.Linear(2048, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride=stride, downsample=True))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [6]:
def load_cifar10_batches():
    train_data = []
    train_labels = []
    
    for i in range(1, 6):  # data_batch_1 to data_batch_5
        batch_file = os.path.join(data_dir, f"data_batch_{i}")
        batch_dict = unpickle(batch_file)
        
        batch_data = batch_dict[b'data']  # Image data (10000, 3072)
        batch_labels = batch_dict[b'labels']  # Labels (10000,)
        
        train_data.append(batch_data)
        train_labels.extend(batch_labels)
    
    train_data = np.vstack(train_data).reshape(-1, 3, 32, 32).astype(np.float32) / 255.0  # Normalize
    train_labels = np.array(train_labels)
    
    return train_data, train_labels

In [7]:
def load_cifar10_test():
    test_file = os.path.join(data_dir, "test_batch")
    test_dict = unpickle(test_file)
    
    test_data = test_dict[b'data'].reshape(-1, 3, 32, 32).astype(np.float32) / 255.0
    test_labels = np.array(test_dict[b'labels'])
    
    return test_data, test_labels

In [8]:
class CIFAR10Dataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx], self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

In [13]:
def test(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    correct, total = 0, 0
    
    with torch.no_grad():  # Disable gradient computation
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)  # Get predicted class
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_acc = 100 * correct / total
    print(f"🎯 Test Accuracy: {test_acc:.2f}%")
    return test_acc

In [9]:
data_dir = "./data/cifar-10-batches-py"

In [10]:
# Load Data
train_data, train_labels = load_cifar10_batches()
test_data, test_labels = load_cifar10_test()

# Split Training Set into Training & Validation (80% Train, 20% Validation)
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_data, val_data = train_data[:train_size], train_data[train_size:]
train_labels, val_labels = train_labels[:train_size], train_labels[train_size:]

# Define Data Transformations (Data Augmentation)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.Normalize((0.5,), (0.5,))
])

# Create Dataset & DataLoader
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=transform)
val_dataset = CIFAR10Dataset(val_data, val_labels, transform=transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=transform)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Loaded CIFAR-10: {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test samples.")

Loaded CIFAR-10: 40000 train, 10000 val, 10000 test samples.


In [11]:
model = CustomResNet().to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_val_acc = 0.0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"\n🔄 Epoch {epoch+1}/{num_epochs} ---------------------------")
    
    ### TRAINING PHASE ###
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Track loss
        running_loss += loss.item()
        
        # Track accuracy
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Print loss for every 50 batches
        if (batch_idx + 1) % 50 == 0 or (batch_idx + 1) == len(train_loader):
            print(f"🟢 [Batch {batch_idx+1}/{len(train_loader)}] Loss: {loss.item():.4f}")

    # Calculate training accuracy
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    print(f"✅ Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%")

    ### VALIDATION PHASE ###
    model.eval()
    correct, total = 0, 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_acc = 100 * correct / total
    print(f"🔵 Validation Accuracy: {val_acc:.2f}%")

    ### SAVE BEST MODEL ###
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print(f"💾 Best model saved with Validation Accuracy: {best_val_acc:.2f}%")

print("\n🎉 Training Completed!")



🔄 Epoch 1/10 ---------------------------
🟢 [Batch 50/313] Loss: 1.7790
🟢 [Batch 100/313] Loss: 1.6734
🟢 [Batch 150/313] Loss: 1.5855
🟢 [Batch 200/313] Loss: 1.6657
🟢 [Batch 250/313] Loss: 1.3360
🟢 [Batch 300/313] Loss: 1.3306
🟢 [Batch 313/313] Loss: 1.3427
✅ Training Loss: 1.7036, Training Accuracy: 38.93%
🔵 Validation Accuracy: 48.29%
💾 Best model saved with Validation Accuracy: 48.29%

🔄 Epoch 2/10 ---------------------------
🟢 [Batch 50/313] Loss: 1.2758
🟢 [Batch 100/313] Loss: 1.3269
🟢 [Batch 150/313] Loss: 1.3842
🟢 [Batch 200/313] Loss: 1.3460
🟢 [Batch 250/313] Loss: 1.2879
🟢 [Batch 300/313] Loss: 0.9574
🟢 [Batch 313/313] Loss: 0.9980
✅ Training Loss: 1.1874, Training Accuracy: 57.40%
🔵 Validation Accuracy: 58.35%
💾 Best model saved with Validation Accuracy: 58.35%

🔄 Epoch 3/10 ---------------------------
🟢 [Batch 50/313] Loss: 1.1122
🟢 [Batch 100/313] Loss: 0.9712
🟢 [Batch 150/313] Loss: 0.9172
🟢 [Batch 200/313] Loss: 1.1087
🟢 [Batch 250/313] Loss: 0.7813
🟢 [Batch 300/313] Loss

NameError: name 'test' is not defined

In [14]:
# Test
model.load_state_dict(torch.load("best_model.pth"))
model.to(device)
model.eval()
test(model, test_loader, device)

🎯 Test Accuracy: 84.17%


84.17

In [15]:
# Function to count model parameters
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"🔢 Total Trainable Parameters: {total_params:,}")  # Print with commas
    return total_params

# Check model parameters
total_params = count_parameters(model)

# Check if model meets the requirement
if total_params > 5_000_000:
    print("❌ Model exceeds 5 million parameters! Consider reducing layers or channels.")
else:
    print("✅ Model meets the requirement (≤5 million parameters).")


🔢 Total Trainable Parameters: 11,193,546
❌ Model exceeds 5 million parameters! Consider reducing layers or channels.


In [16]:
from torchsummary import summary

# Move model to the correct device (CPU or GPU)
model.to(device)

# Print model summary (assuming input image size is 3×32×32 for CIFAR-10)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]           4,096
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
            Conv2d-7           [-1, 64, 32, 32]          36,864
       BatchNorm2d-8           [-1, 64, 32, 32]             128
     ResidualBlock-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
           Conv2d-12           [-1, 64, 32, 32]          36,864
      BatchNorm2d-13           [-1, 64, 32, 32]             128
    ResidualBlock-14           [-1, 64,